# Check Metadata EEG - Impedances
Status: Implemented
 -- Fully automatized

In [1]:
from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
eeg_files = get_eeg_files_per_participant()
assert (set(folders.keys()) == set(eeg_files.keys()))

### Check Impedance level of electrodes
* for 4 EOG: less than 10 kOhm
* for others: less than 5 kOhm

max. 2 errors allowed

In [3]:
# Iterate over all participants to check
impedance_data_all = {}


for participant in tqdm(folders):

    print('----------------------------------------------')
    print(PARTICIPANT, participant)
    eeg_file = eeg_files[participant]

    print('(01/06) Load EEG data')
    raw_eeg = mne.io.read_raw_brainvision(
        eeg_file[EEG_FILE_HEADER_ENDING], EOG_CHANNELS, preload=False)
    sfreq = raw_eeg.info[MNE_KEY_FREQUENCY]

    print('(02/06) Load impedances')
    impedances = raw_eeg.impedances
    if not impedances:
        print(
            f'\tImpedances cannot be found and extracted automatically from file {eeg_file[EEG_FILE_HEADER_ENDING]}. Please check manually.')
        continue
    impedance_data = pd.DataFrame.from_dict(raw_eeg.impedances, orient='index')

    print('(03/06) Set boundaries for impedance values')
    # EOG less than 10, others less than 5
    impedance_data[IMPEDANCE_UPPER_BOUND] = 5
    for eog_c in EOG_CHANNELS:
        impedance_data.at[eog_c, IMPEDANCE_UPPER_BOUND] = 10

    print('(04/06) Check impedances')
    # check which channel impedances exceed the upper boundary
    impedance_okay = check_impedances(impedance_data, log=True)
    if not impedance_okay:
        print('(05/06) Update exclusions')
        update_exclusions(participant, PARTICIPANT, EEG, {
                          '': '01b_PRE_EEG_Metadata_Check: Impedance not acceptable.'})

    print('(06/06) Save impedances')
    impedance_data[PARTICIPANT] = participant
    impedance_data_all[participant] = impedance_data
    impedance_data.to_csv(get_impedance_path(participant), sep=SEPARATOR)

impedance_data = pd.concat(impedance_data_all.values())
impedance_data.to_csv(get_impedance_path(), sep=SEPARATOR)

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
(01/06) Load EEG data
Extracting parameters from ..\08-Data-Trial_Recordings\raw\E003\AoCfrp_003.vhdr...
Setting channel info structure...
(02/06) Load impedances
(03/06) Set boundaries for impedance values
(04/06) Check impedances
2 errors with impedances, maximum breach of 3.0, sum of breaches in total 4.0.
	Impedances accepted: True
(06/06) Save impedances
----------------------------------------------
Participant 004
(01/06) Load EEG data
Extracting parameters from ..\08-Data-Trial_Recordings\raw\E004\AoCfrp_004.vhdr...
Setting channel info structure...
(02/06) Load impedances
(03/06) Set boundaries for impedance values
(04/06) Check impedances
0 errors with impedances, maximum breach of 0, sum of breaches in total 0.
	Impedances accepted: True
(06/06) Save impedances
----------------------------------------------
Participant 005
(01/06) Load EEG data
Extracting parameters from ..\08-Data-Trial_Recordings\raw\E005\AoCfr